# (WARNING) 8 GPUs with a total VRAM of 144 GB required to train

In [ ]:
!rm -rf detectron2/
!python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'

In [ ]:
from datetime import datetime
import logging

from detectron2.checkpoint import DetectionCheckpointer
from detectron2.config import LazyConfig, instantiate
from detectron2.engine import (
    AMPTrainer,
    SimpleTrainer,
    default_argument_parser,
    default_setup,
    default_writers,
    hooks,
    launch
)
from detectron2.engine.defaults import _try_get_key
from detectron2.engine.defaults import create_ddp_model
from detectron2.evaluation import inference_on_dataset, print_csv_format
from detectron2.utils import comm

from functools import partial
from detectron2.utils.file_io import PathManager
from omegaconf import OmegaConf
import torch.nn as nn
from fvcore.common.param_scheduler import MultiStepParamScheduler

from detectron2 import model_zoo
from detectron2.config import LazyCall as L
from detectron2.solver import WarmupParamScheduler
from detectron2.modeling import MViT
from detectron2.layers import ShapeSpec
from detectron2.modeling.box_regression import Box2BoxTransform
from detectron2.modeling.matcher import Matcher
from detectron2.modeling.roi_heads import (
    FastRCNNOutputLayers,
    FastRCNNConvFCHead,
    CascadeROIHeads,
)
from detectron2.utils.env import seed_all_rng
from detectron2.data.datasets import register_coco_instances
import detectron2.data.transforms as T
from detectron2 import model_zoo
from detectron2.config import LazyCall as L
import detectron2.data.transforms as T
from detectron2.config import LazyCall as L
from detectron2.data import (
    DatasetMapper,
    build_detection_test_loader,
    build_detection_train_loader,
    get_detection_dataset_dicts,
)
from detectron2.evaluation import COCOEvaluator
from detectron2.utils.logger import setup_logger
from detectron2.utils.collect_env import collect_env_info

In [ ]:
def setup():
    model = model_zoo.get_config("common/models/mask_rcnn_fpn.py").model
    constants = model_zoo.get_config("common/data/constants.py").constants
    model.pixel_mean = constants.imagenet_rgb256_mean
    model.pixel_std = constants.imagenet_rgb256_std
    model.input_format = "RGB"
    model.backbone.bottom_up = L(MViT)(
        embed_dim=96,
        depth=24,
        num_heads=1,
        last_block_indexes=(1, 4, 20, 23),
        residual_pooling=True,
        drop_path_rate=0.4,
        norm_layer=partial(nn.LayerNorm, eps=1e-6),
        out_features=("scale2", "scale3", "scale4", "scale5"),
    )
    model.backbone.in_features = "${.bottom_up.out_features}"
    model.backbone.square_pad = 1024

    # New heads and LN
    model.backbone.norm = "LN"  # Use LN in FPN
    model.roi_heads.box_head.conv_norm = model.roi_heads.mask_head.conv_norm = "LN"

    # 2conv in RPN:
    model.proposal_generator.head.conv_dims = [-1, -1]

    # arguments that don't exist for Cascade R-CNN
    [model.roi_heads.pop(k) for k in ["box_head", "box_predictor", "proposal_matcher"]]
    model.roi_heads.update(
        _target_=CascadeROIHeads,
        box_heads=[
            L(FastRCNNConvFCHead)(
                input_shape=ShapeSpec(channels=256, height=7, width=7),
                conv_dims=[256, 256, 256, 256],
                fc_dims=[1024],
                conv_norm="LN",
            )
            for _ in range(3)
        ],
        box_predictors=[
            L(FastRCNNOutputLayers)(
                input_shape=ShapeSpec(channels=1024),
                test_score_thresh=0.05,
                box2box_transform=L(Box2BoxTransform)(weights=(w1, w1, w2, w2)),
                cls_agnostic_bbox_reg=True,
                num_classes="${...num_classes}",
                test_topk_per_image=1000
            )
            for (w1, w2) in [(10, 5), (20, 10), (30, 15)]
        ],
        proposal_matchers=[
            L(Matcher)(thresholds=[th], labels=[0, 1], allow_low_quality_matches=False)
            for th in [0.5, 0.6, 0.7]
        ],
    )
    model.roi_heads.num_classes = 4
    model.roi_heads.batch_size_per_image = 512

    dataloader = OmegaConf.create()

    image_size = 1024
    dataloader.train = L(build_detection_train_loader)(
        dataset=L(get_detection_dataset_dicts)(names="badlad_train"),
        mapper=L(DatasetMapper)(
            is_train=True,
            augmentations=[
                L(T.RandomBrightness)(intensity_min=0.8,intensity_max=1.2),
                L(T.RandomContrast)(intensity_min=0.5,intensity_max=1.5),
                L(T.RandomSaturation)(intensity_min=0.5,intensity_max=1.0),
                L(T.RandomRotation)(angle=[-5, 5], sample_style="range"),
                L(T.ResizeScale)(
                    min_scale=0.1, max_scale=2.0, target_width=image_size, target_height=image_size
                ),
                L(T.FixedSizeCrop)(crop_size=(image_size, image_size), pad=False),
            ],
            image_format="RGB",
            use_instance_mask=True,
        ),
        total_batch_size=16,
        num_workers=4,
    )

    dataloader.test = L(build_detection_test_loader)(
        dataset=L(get_detection_dataset_dicts)(names="badlad_test", filter_empty=False),
        mapper=L(DatasetMapper)(
            is_train=False,
            augmentations=[
                L(T.ResizeShortestEdge)(short_edge_length=image_size, max_size=image_size),
            ],
            image_format="RGB",
        ),
        batch_size=8,
        num_workers=2,
    )

    dataloader.evaluator = L(COCOEvaluator)(
        dataset_name="${..test.dataset.names}",
    )

    dataloader.train.num_workers = 2
    dataloader.train.total_batch_size = 16
    # recompute boxes due to cropping
    dataloader.train.mapper.recompute_boxes = True

    # Initialization and trainer settings
    train = model_zoo.get_config("common/train.py").train
    train.amp.enabled = True
    train.ddp.fp16_compression = True
    train.init_checkpoint = "detectron2://ImageNetPretrained/mvitv2/MViTv2_B_in21k.pyth"
    train.output_dir = "./output/mvit2b"
    # Schedule
    # 36 epoch = 20365/16 * 36 = 45821 iterations 
    train.max_iter = 45821
    train.eval_period = 1000
    train.log_period = 20
    train.checkpointer.period = 2000
    train.device = "cuda"

    lr_multiplier = L(WarmupParamScheduler)(
        scheduler=L(MultiStepParamScheduler)(
            values=[1.0, 0.1, 0.01],
            milestones=[40730, 44447],
            num_updates=train.max_iter,
        ),
        warmup_length=50 / train.max_iter,
        warmup_factor=0.001,
    )

    optimizer = model_zoo.get_config("common/optim.py").AdamW
    optimizer.params.overrides = {"pos_embed": {"weight_decay": 0.0}}
    optimizer.lr = 0.00008

    dataloader.evaluator.output_dir = train.output_dir

    cfg = OmegaConf.create()
    cfg.model = model
    cfg.dataloader = dataloader
    cfg.train = train
    cfg.optimizer = optimizer
    cfg.lr_multiplier = lr_multiplier

    return cfg

In [ ]:
def do_test(cfg, model):
    if "evaluator" in cfg.dataloader:
        ret = inference_on_dataset(
            model, instantiate(cfg.dataloader.test), instantiate(cfg.dataloader.evaluator)
        )
        print_csv_format(ret)
        return ret

In [ ]:
def do_train(resume, cfg):
    model = instantiate(cfg.model)
    logger = logging.getLogger("detectron2")
    logger.info("Model:\n{}".format(model))
    model.to(cfg.train.device)

    cfg.optimizer.params.model = model
    optim = instantiate(cfg.optimizer)

    train_loader = instantiate(cfg.dataloader.train)

    model = create_ddp_model(model, **cfg.train.ddp)
    trainer = (AMPTrainer if cfg.train.amp.enabled else SimpleTrainer)(model, train_loader, optim)
    checkpointer = DetectionCheckpointer(
        model,
        cfg.train.output_dir,
        trainer=trainer,
    )
    trainer.register_hooks(
        [
            hooks.IterationTimer(),
            hooks.LRScheduler(scheduler=instantiate(cfg.lr_multiplier)),
            hooks.PeriodicCheckpointer(checkpointer, **cfg.train.checkpointer)
            if comm.is_main_process()
            else None,
            # hooks.EvalHook(cfg.train.eval_period, lambda: do_test(cfg, model)),
            hooks.PeriodicWriter(
                default_writers(cfg.train.output_dir, cfg.train.max_iter),
                period=cfg.train.log_period,
            )
            if comm.is_main_process()
            else None,
        ]
    )

    checkpointer.resume_or_load(cfg.train.init_checkpoint, resume=resume)
    if resume and checkpointer.has_checkpoint():
        start_iter = trainer.iter + 1
    else:
        start_iter = 0
    trainer.train(start_iter, cfg.train.max_iter)

In [ ]:
def main():
    cfg=setup()
    output_dir = cfg.train.output_dir
    if comm.is_main_process() and output_dir:
        PathManager.mkdirs(output_dir)

    rank = comm.get_rank()
    setup_logger(output_dir, distributed_rank=rank, name="fvcore")
    logger = setup_logger(output_dir, distributed_rank=rank)

    logger.info("Rank of current process: {}. World size: {}".format(rank, comm.get_world_size()))
    logger.info("Environment info:\n" + collect_env_info())


    # make sure each worker has a different, yet deterministic seed if specified
    cfg.train.seed = int(datetime.now().timestamp()) + rank

    do_train(False, cfg)

if __name__ == "__main__":
    launch(
        main,
        8, # train on 8 GPUs
        num_machines=1,
        args=(),
    )